# Lower Thwaites Glacier

<p align=center>
<img src="http://gnss-reflections.org/static/images/LTHW.jpg" width="500"/><BR>
</p>

**Station Name:** lthw

**Location:** Thwaites Glacier, Antarctica

**Archive:** [EarthScope Consortium](http://www.earthscope.org)

**Ellipsoidal Coordinates:**

- Latitude: -76.458
- Longitude: -107.782
- Height: 1011.459 m

[Station Page at EarthScope](https://www.unavco.org/instrumentation/networks/status/nota/overview/LTHW)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/LTHW.sta)


In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

## Data Summary

The receiver only tracks legacy GPS signals, so only L1 should be used for reflectometry. The pole
is set in the snow/ice and routinely (every few years) reset. Please 
use the [Nevada Reno site](http://geodesy.unr.edu/NGLStationPages/stations/LTHW.sta) to get a 
feel for when the pole has been reset and where data gaps exist.

Because there are no permanent structures surrounding the site, elevation and azimuth angle default settings can mostly be used.
The only restriction that should be imposed is a minimum elevation angle of 7; this is because the field
crew set this at the receiver when it was originally installed.

lthw is one of the example cases for the [GNSS-IR webapp.](https://gnss-reflections.org/api?example=lthw) 

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=lthw" width="800" height="950"></iframe>

## Take a quick look at the SNR data

Translate the GPS data for January 1 in 2018. First you need to make the SNR file:

In [ ]:
station = "lthw"
year = 2018
doy = 1

In [ ]:
rinex2snr(station, year, doy)

Use our utility `quickLook` to look at these data [(For more details on quickLook output)](../pages/quickLook.md):

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

This is a bit of a mess really. If there are significant peaks, they are really 
close to the cutoff for the method (at 0.5 meters). Let's compare with about a week later.
First make a SNR file:

In [ ]:
rinex2snr(station, year=year, doy=9)

values, metrics = quicklook(station, year, doy=9)

This is *much* better and clearly shows that a field crew reset the antenna to a little 
less than 5 meters sometime between day 1 and day 9 in the year 2018. The second plot to gives an idea of how well the method performs for various azimuths and QC parameters.

Except for a region ~270-300 degrees in azimuth, and directly south, retrievals are consistently good.
A peak to noise ratio of 3 appears to be appropriate. The rejected southern retrievals are unlikely to be 
of local origin, but rather related to the inclination of the satellite orbits at this latitude.

Lets compare this to the first day of 2020:

In [ ]:
rinex2snr(station, year=2020, doy=1)

values, metrics = quicklook(station, year=2020, doy=1)

Now the peaks in the reflector height (RH) periodograms are ~2.2 meters - 
so that means that there was ~2.5 meters of surface change from 2018 to 2020.

## Measure Snow Accumulation for 2018

Translate the GPS data for the year of 2018:

In [ ]:
rinex2snr(station, year, 1, doy_end=366, weekly=weekly.value)

Then you need to make the list of analysis inputs (stored in json format):

In [ ]:
make_gnssir_input(station, e1=7, peak2noise=3.2, l1=True)

It is fine to hand edit the json file to remove the unreliable azimuths if 
you prefer.

Now analyze the data for 2018 from day 1 to day 365 using **gnssir**:

In [ ]:
gnssir(station, year, doy, doy_end=366)

This produces reflector heights for every rising and setting satellite track that meets your 
quality control selections.  In order to estimate snow accumulation, you will want to calculate
the daily average. Using our **daily_avg** utility - and specifying 50 satellite tracks and median filter of 0.25 meters:

In [ ]:
daily_avg(station, medfilter=0.25, ReqTracks=50)

You can loosen the required track number if you want, but in this case it does not change the 
overall character of the results:

In [ ]:
daily_avg(station, medfilter=0.25, ReqTracks=40)

If you are feeling ambitious, try looking at the data for [Upper Thwaites Glacier](https://www.unavco.org/instrumentation/networks/status/nota/overview/UTHW)